In [ ]:
# | default_exp routes.page

In [ ]:
# | exporti
import httpx
from nbdev import show_doc

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

# Route Errors

In [ ]:
#| export

class PageRetrieval_byId_Error(de.DomoError):
    def __init__(self, status, domo_instance, page_id, response, function_name = None, parent_class = None):
        super().__init__(
            status = status,
            function_name = function_name,
            parent_class = parent_class,
            message = f"failed to retrieve page_id: {page_id}"
        )

#### sample implementation of PageRetrieval_byId_Error

In [ ]:
#| eval: false
try:
    raise PageRetrieval_byId_Error(
        status=404,
        page_id=123,
        function_name="test",
        parent_class="Foo",
        domo_instance="test_domo",
        response="Bad Request",
    )
except PageRetrieval_byId_Error as e:
    print(e)

🛑  PageRetrieval_byId_Error 🛑 - functionn: Foo.test || status 404 || failed to retrieve page_id: 123


In [ ]:
# | export        
async def get_page_by_id(
    auth: dmda.DomoAuth,
    page_id: str,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    include_layout: bool = False, # passes parameter to return page layout information
    debug_num_stacks_to_drop : int = 1, # for traceback_details.  use 1 for route functions, 2 for class method
    parent_class : str = None # pass in self.__class__.__name__ into function
) -> rgd.ResponseGetData: # returns ResponseGetData on success or raise Exception on error
 
    """retrieves a page or throws an error"""

    # 9/21/2023 - the domo UI uses /cards to get page info
    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/stacks/{page_id}/cards" 

    if include_layout:
        url += "?includeV4PageLayouts=true"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        num_stacks_to_drop = debug_num_stacks_to_drop,
        parent_class = parent_class
    )

    if not res.is_success or not isinstance(res.response, dict) or not res.response.get('id', None):
        raise PageRetrieval_byId_Error(
            status=res.status,
            page_id=page_id,
            function_name=res.traceback_details.function_name,
            parent_class= parent_class,
            domo_instance= auth.domo_instance,
            response=res.response,
        )

    return res

In [ ]:
show_doc(get_page_by_id)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/page.py#L27){target="_blank" style="float:right; font-size:smaller"}

### get_page_by_id

>      get_page_by_id (auth:domolibrary.client.DomoAuth.DomoAuth, page_id:str,
>                      debug_api:bool=False, session:httpx.AsyncClient=None,
>                      include_layout:bool=False,
>                      debug_num_stacks_to_drop:int=1, parent_class:str=None)

retrieves a page or throws an error

#### sample get_page_by_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366

res = await get_page_by_id(page_id=page_id, auth=token_auth)

from pprint import pprint
pprint(res.response)

{'cards': [{'active': True,
            'allowTableDrill': True,
            'badgeUpdated': 1631619767000,
            'created': 1631619158,
            'creatorId': 1898323170,
            'description': '',
            'id': 1548948000,
            'locked': False,
            'metadata': {'SummaryNumberFormat': '{"type":"number","format":"#A"}',
                         'calendar': 'default',
                         'chartType': 'badge_vert_multibar',
                         'chartVersion': '8',
                         'columnAliases': '{}',
                         'columnFormats': '{}',
                         'currentLabel': '',
                         'currentMethod': 'empty',
                         'historyId': '6fa216c4-1465-4242-917c-3ca9e018c8be'},
            'ownerId': 1898323170,
            'title': 'View of 75th Percentile Test - Baseball Stats',
            'type': 'kpi',
            'urn': '1548948000'},
           {'active': True,
            'allowTableDril

In [ ]:
# | export
async def get_page_definition(
    auth : dmda.DomoAuth,
    page_id : int,
    debug_api: bool = False, 
    session: httpx.AsyncClient = None,
    parent_class : str = None,
    debug_num_stacks_to_drop : int = 1
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/stacks/{page_id}/cards"

    params = {
        "includeV4PageLayouts": "true",
        "parts": "metadata,datasources,library,drillPathURNs,certification,owners,dateInfo,subscriptions,slicers",
    }

    res = await gd.get_data(
        url,
        method="GET",
        auth=auth,
        session=session,
        params=params,
        debug_api=debug_api,
        num_stacks_to_drop = debug_num_stacks_to_drop,
        parent_class = parent_class
    )

    if not res.is_success or not isinstance(res.response, dict) or not res.response.get('id', None):
        raise PageRetrieval_byId_Error(
            status=res.status,
            page_id=page_id,
            function_name=res.traceback_details.function_name,
            parent_class= parent_class,
            domo_instance= auth.domo_instance,
            response=res.response,
        )

    return res

In [ ]:
show_doc(get_page_definition)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/page.py#L68){target="_blank" style="float:right; font-size:smaller"}

### get_page_definition

>      get_page_definition (auth:domolibrary.client.DomoAuth.DomoAuth,
>                           page_id:int, debug_api:bool=False,
>                           session:httpx.AsyncClient=None,
>                           parent_class:str=None,
>                           debug_num_stacks_to_drop:int=1)

#### sample implementation of get_page_definition


In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366

res = await get_page_definition(page_id=page_id, auth=token_auth)

res.response

{'id': '1761849366',
 'page': {'pageId': 1761849366,
  'owners': [{'id': 592838897,
    'type': 'GROUP',
    'displayName': 'Test Group ABC'}],
  'type': 'page',
  'title': 'delete me',
  'pageName': 'delete me',
  'created': 1625863915000,
  'updated': 1695648708568,
  'locked': False,
  'mobileEnabled': True,
  'sharedViewPage': True,
  'virtualPage': False,
  'isOwner': True},
 'type': 'page',
 'title': 'delete me',
 'sizes': [{'id': '2056437956', 'size': ''}, {'id': '1548948000', 'size': ''}],
 'cards': [{'metadata': {'chartType': 'badge_vert_multibar',
    'chartVersion': '8',
    'currentLabel': '',
    'currentMethod': 'empty',
    'historyId': '2e5a1be4-8d73-4872-9ce1-20151ccd7e31',
    'calendar': 'default',
    'columnAliases': '{}',
    'columnFormats': '{}',
    'SummaryNumberFormat': '{"type":"number","format":"#A"}'},
   'drillPathURNs': [],
   'subscriptions': [{'cardId': 2056437956,
     'dataSourceId': '4ef43af5-67e7-4b9e-bd58-c4e592aa289a',
     'dataSourceName': 'Vie

In [ ]:
# | export

async def test_page_access(
    auth,
    page_id,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1
):
    """retrieves accesslist, which users and groups a page is shared with"""
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/pages/{page_id}/access"

    res = await gd.get_data(
        url, method="GET", auth=auth, session=session, debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class
    )
    
    return res


#### sample implementation test_page_access

In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await test_page_access(page_id=page_id,
                             auth=token_auth,
                             )

# for user in res.response.get('users'):
#     print({user.get('displayName')})

pprint(res.response)


{'owners': [{'displayName': 'Test Group ABC',
             'id': 592838897,
             'type': 'GROUP'}],
 'pageAccess': True,
 'pageId': 1761849366,
 'pageName': 'delete me'}


In [ ]:
# | export
async def get_page_access_list(
    auth,
    page_id,
    is_expand_users: bool = True,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1
):
    """retrieves accesslist, which users and groups a page is shared with"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/share/accesslist/page/{page_id}?expandUsers={is_expand_users}"

    res = await gd.get_data(
        url, method="GET", auth=auth, session=session, debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class
    )

    res.response['explicitSharedUserCount'] = len(res.response.get('users'))
    for user in res.response.get('users'):
        user.update({'isExplicitShare': True})

    # add group members to users response
    if is_expand_users:

        group_users = [
            {**user,
            'isExplicitShare' : False
            } for group in res.response.get("groups") for user in group.get("users")
        ]
        users = res.response.get("users") + [group_user for group_user in group_users if group_user.get('id') not in [user.get('id') for user in res.response.get("users")]]
        res.response.update({"users": users})

    res.response['expandUserCount'] = len(res.response.get('users'))

    return res


#### sample get_page_accesslist


In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_access_list(page_id=page_id,
                                 auth=token_auth,
                                 is_expand_users=True)

# for user in res.response.get('users'):
#     print({user.get('displayName')})

pprint(res.response)


{'expandUserCount': 5,
 'explicitSharedUserCount': 1,
 'groups': [{'active': True,
             'creatorId': 1893952720,
             'default': False,
             'description': '',
             'hidden': False,
             'id': 592838897,
             'name': 'Test Group ABC',
             'type': 'open',
             'users': [{'accepted': True,
                        'active': True,
                        'anonymous': False,
                        'created': 1655998912,
                        'displayName': 'Aaron Dean',
                        'emailAddress': 'aaron.dean@rxa.io',
                        'id': 1628021317,
                        'invitorUserId': 617808774,
                        'modified': 1693590493425,
                        'pending': False,
                        'roleId': 2097317660,
                        'systemUser': False,
                        'userName': 'aaron.dean@rxa.io',
                        'userType': 'USER'},
                     

In [ ]:
# |export
async def get_pages_adminsummary(
    auth: dmda.DomoAuth,
    debug_loop: bool = False,
    debug_api: bool = False,
    limit=35,
    session: httpx.AsyncClient = None,
):
    """retrieves all pages in instance user is able to see (but may not have been explicitly shared)"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/pages/adminsummary"

    offset_params = {
        "offset": "skip",
        "limit": "limit",
    }

    body = {"orderBy": "pageTitle", "ascending": True}

    def arr_fn(res) -> list[dict]:
        return res.response.get("pageAdminSummaries")

    res = await gd.looper(
        auth=auth,
        method="POST",
        url=url,
        arr_fn=arr_fn,
        offset_params=offset_params,
        session=session,
        loop_until_end=True,
        body=body,
        limit=limit,
        debug_loop=debug_loop,
        debug_api=debug_api,
    )
    return res

In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_pages_adminsummary(auth=token_auth, debug_loop=False, debug_api=False)

ResponseGetData(status=200, response=[{'pageId': 1316566624, 'pageTitle': '20210623_TRAINING_DomoStats Activity Log App', 'pageType': 'page', 'parentPageId': 127044793, 'parentPageTitle': 'JaeW_AtOnyx', 'topPageId': 522373865, 'topPageTitle': 'Learn Domo', 'ownerId': 1893952720, 'ownerName': 'Jae Wilson1', 'owners': [{'id': 1893952720, 'type': 'USER', 'displayName': 'Jae Wilson1'}], 'locked': False, 'lastModified': 1624448913807, 'cardCount': 34, 'dataAppId': None, 'dataAppTitle': None}, {'pageId': 384424178, 'pageTitle': '75th Percentile Test', 'pageType': 'page', 'parentPageId': None, 'parentPageTitle': None, 'topPageId': None, 'topPageTitle': None, 'ownerId': 1898323170, 'ownerName': 'Creed Smith', 'owners': [{'id': 1898323170, 'type': 'USER', 'displayName': 'Creed Smith'}], 'locked': False, 'lastModified': 1681764847029, 'cardCount': 6, 'dataAppId': None, 'dataAppTitle': None}, {'pageId': 1917664953, 'pageTitle': 'A', 'pageType': 'page', 'parentPageId': 1603030697, 'parentPageTitle

In [ ]:
# |export
async def update_page_layout(
    auth: dmda.DomoAuth, layout_id: str, body: dict, debug_api: bool = False
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v4/pages/layouts/{layout_id}"

    res = await gd.get_data(
        auth=auth, url=url, body=body, method="PUT", debug_api=debug_api
    )

    if debug_api:
        print(res)

    return res


async def put_writelock(
    auth: dmda.DomoAuth,
    layout_id: str,
    user_id: str,
    epoch_time: int,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v4/pages/layouts/{layout_id}/writelock"
    body = {
        "layoutId": layout_id,
        "lockHeartbeat": epoch_time,
        "lockTimestamp": epoch_time,
        "userId": user_id,
    }

    res = await gd.get_data(
        auth=auth, url=url, body=body, method="PUT", debug_api=debug_api
    )

    if debug_api:
        print(res)

    return res


async def delete_writelock(
    auth: dmda.DomoAuth, layout_id: str, debug_api: bool = False
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v4/pages/layouts/{layout_id}/writelock"
    res = await gd.get_data(auth=auth, url=url, method="DELETE", debug_api=debug_api)

    if debug_api:
        print(res)

    return res

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()